# Logistic Regression

- Date: 2018-04-12

In [1]:
# Import libraries

import torch          
import torch.nn as nn  
import torchvision.datasets as dsets  
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
# Hyper-parameters

input_size=784
num_classes=10
num_epochs=5
batch_size=100
learning_rate=1e-3

In [19]:
# MNIST Dataset (Images and labels)

train_dataset = dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.MNIST(root='./data', train=False, transform=transforms.ToTensor())
print(len(train_dataset))
print(len(test_dataset))

60000
10000


In [16]:
# Data Loader(Input Pipeline)

train_loader=torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
# Model

class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear=nn.Linear(input_size, num_classes)
    def forward(self, x):
        out=self.linear(x)
        return out


In [7]:
model=LogisticRegression(input_size, num_classes)

In [9]:
# Loss and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [20]:
# Train the model

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images) # prediction. forward
        loss = criterion(outputs, labels) # prediction, real value
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, len(train_dataset)/batch_size, loss.data[0]))

            
# Mini-batch 할 때 Tip, 순서
# zero_grad()를 하지 않으면 gradient가 계속해서 accumulate 된다. 고로..
# Zero_Grad -> Forward -> Backward -> Step -> Repeat

# loss.backward()
# gradient가 accumulate되는 과정. 그래서 여러번 하면 계속 accumulate되므로 grad=0로 만든 후에, 다시 compute forward => backward 해줘야 함.

# optimizer.step() = 쉽게, parameter를 update한다고 생각하면 됨.
# performs a parameter update based on current gradient 

Epoch: [1/5], Step: [100/600], Loss: 0.6173
Epoch: [1/5], Step: [200/600], Loss: 0.5657
Epoch: [1/5], Step: [300/600], Loss: 0.6644
Epoch: [1/5], Step: [400/600], Loss: 0.4157
Epoch: [1/5], Step: [500/600], Loss: 0.4765
Epoch: [1/5], Step: [600/600], Loss: 0.6376
Epoch: [2/5], Step: [100/600], Loss: 0.5463
Epoch: [2/5], Step: [200/600], Loss: 0.5289
Epoch: [2/5], Step: [300/600], Loss: 0.4857
Epoch: [2/5], Step: [400/600], Loss: 0.5720
Epoch: [2/5], Step: [500/600], Loss: 0.4738
Epoch: [2/5], Step: [600/600], Loss: 0.4858
Epoch: [3/5], Step: [100/600], Loss: 0.5090
Epoch: [3/5], Step: [200/600], Loss: 0.4695
Epoch: [3/5], Step: [300/600], Loss: 0.5769
Epoch: [3/5], Step: [400/600], Loss: 0.5690
Epoch: [3/5], Step: [500/600], Loss: 0.4691
Epoch: [3/5], Step: [600/600], Loss: 0.4677
Epoch: [4/5], Step: [100/600], Loss: 0.4403
Epoch: [4/5], Step: [200/600], Loss: 0.5641
Epoch: [4/5], Step: [300/600], Loss: 0.4988
Epoch: [4/5], Step: [400/600], Loss: 0.5919
Epoch: [4/5], Step: [500/600], L

In [42]:
# Test the model

correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = model(images) # prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    #print(labels.size(0)) # 100 나옴 
    correct += (predicted==labels).sum()

print('Accuracy of the model on the 10,000 test images: %d %%' %(100*correct/total))

Accuracy of the model on the 10,000 test images: 88 %


In [41]:
# Save the Model

torch.save(model.state_dict(), 'model.pkl')